In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, Rescaling
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.utils import image_dataset_from_directory
import pathlib

import pickle

import matplotlib.pyplot as plt

In [29]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4901455200000228266]

In [3]:
# 폴더에서 데이터 읽기
data_path = pathlib.Path("./datasets/stanford_dogs/images/images")

train_ds = image_dataset_from_directory(data_path, validation_split=0.2, subset="training", seed=123, image_size=(224,224), batch_size=16)
test_ds = image_dataset_from_directory(data_path, validation_split=0.2, subset="validation", seed=123, image_size=(224,224), batch_size=16)

Found 20580 files belonging to 120 classes.
Using 16464 files for training.
Found 20580 files belonging to 120 classes.
Using 4116 files for validation.


In [4]:
# 모델 설정
    # DenseNet121을 백본으로 사용
    # weights="imagenet": ImageNet으로 사전 학습된 가중치 로드
    # include_top=False: 모델의 뒤쪽에 있는 완전연결층 제외
    # input_shape=(224,224,3): 신경망 입력 텐서 크기 지정
base_model = DenseNet121(weights="imagenet", include_top=False, input_shape=(224,224,3))
cnn = Sequential()
cnn.add(Rescaling(1.0/255.0)) # 입력텐서를 [0, 1]로 정규화
cnn.add(base_model)
cnn.add(Flatten()) # 백본 출력텐서를 1차원으로 flattening
cnn.add(Dense(1024, activation="relu"))
cnn.add(Dropout(0.75))
cnn.add(Dense(units=120, activation="softmax")) # 결과 출력을 위한 완전연결층, 120개 부류, softmax

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [5]:
# 학습
cnn.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(learning_rate=0.000001), metrics=["accuracy"]) # 미세조정 방식의 전이학습, 학습률을 아주 작게 설정하여 특징추출을 담당하는 층의 가중치를 유지
hist = cnn.fit(train_ds, epochs=200, validation_data=test_ds, verbose=2) # 미세조정을 위해 epoch의 수를 늘려 오래 학습

Epoch 1/200


KeyboardInterrupt: 

In [ ]:
# 예측
print("정확률=", cnn.evaluate(test_ds, verbose=0)[1]*100)

In [ ]:
# 모델 저장
cnn.save("cnn_for_stanford_dogs.h5") # 미세 조정된 모델을 파일에 저장 # [비전에이전트 7]을 위해 모델을 저장

In [ ]:
f = open("dong_species_names.txt", "wb")
pickle.dump(train_ds.class_names, f)
f.close()

In [ ]:
# 정확도 그래프 그리기
plt.plot(hist.history["accuracy"])
plt.plot(hist.history["val_accuracy"])
plt.title("Accuracy graph")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(["Train", "Validation"])
plt.grid()
plt.show()

In [ ]:
# 손실 그래프 그리기
plt.plot(hist.history["loss"])
plt.plot(hist.history["val_loss"])
plt.title("Loss graph")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(["Train", "Validation"])
plt.grid()
plt.show()